<a href="https://colab.research.google.com/github/vilsonrodrigues/DataScienceEduc/blob/master/Tabela_de_submiss%C3%B5es_juntando_com_os_arquivos_do_Sigaa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Total de submissões com ruídos: 37758
Total de submissões com os dados limpos: 36702

Critério de exclusão: se a próxima submissão tiver um intervalo de tempo de 3s de diferença da anterior

Ano turma != ano sigaa então exclua

# Modelagem de dados

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime as dt

Lendo CSV's

In [2]:
sub = pd.read_csv("https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lop_db_2019_09_csv_tratados/lopSubLista%20(1).csv")
sub = sub.iloc[:,[1,2,4,6]]
sub.head()

,data,idAluno,porcentagemAcerto,questao
0,2017-08-14 07:56:26.083,598cd5453bc48946977dcf66,50.0,59887c83684f4a3f78163c02
1,2017-08-14 11:47:22.534,598cd5453bc48946977dcf66,100.0,59888ea5d2f43b3f548ef057
2,2017-08-16 18:02:39.088,598cd5453bc48946977dcf66,100.0,5994485a487ae0183f7dbffa
3,2017-08-16 18:11:55.199,598cd5453bc48946977dcf66,100.0,5994498d0875c31866aa6922
4,2017-08-16 18:36:53.398,598cd5453bc48946977dcf66,100.0,5994434aa76e28186bdf5672


In [3]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47639 entries, 0 to 47638
Data columns (total 4 columns):
data                 47639 non-null object
idAluno              47639 non-null object
porcentagemAcerto    47639 non-null float64
questao              47639 non-null object
dtypes: float64(1), object(3)
memory usage: 1.5+ MB


Passando as datas para datetime

In [4]:
sub["data"] = pd.to_datetime(sub["data"])
sub["data"] = sub["data"].dt.tz_localize(None)
#sub["dia"] = sub["data"].dt.day
sub["data"].head()

0   2017-08-14 07:56:26.083
1   2017-08-14 11:47:22.534
2   2017-08-16 18:02:39.088
3   2017-08-16 18:11:55.199
4   2017-08-16 18:36:53.398
Name: data, dtype: datetime64[ns]

In [5]:
turmas = pd.read_csv("https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lop_db_2019_09_csv_tratados/lopUsuarioTurma%20(1).csv")
turmas.head(2)

,Unnamed: 0,idTurma,ano,descricaoTurma,matricula,idAluno,nome
0,0,598e15296d8650eb27d52e3d,2017.2,LÓGICA DE PROGRAMAÇÃO - Turma 01A,20170106538,599b051d495fac076a06d3f5,Aleff Ramon da Silva Costa
1,1,598e15296d8650eb27d52e3d,2017.2,LÓGICA DE PROGRAMAÇÃO - Turma 01A,20170039453,5991cb910be9d42f2f68b906,Amanda


In [6]:
turmas = turmas.iloc[:,1:]
turmas.rename(columns={"ano":"anoTurma"},inplace=True)
turmas.head(2)

,idTurma,anoTurma,descricaoTurma,matricula,idAluno,nome
0,598e15296d8650eb27d52e3d,2017.2,LÓGICA DE PROGRAMAÇÃO - Turma 01A,20170106538,599b051d495fac076a06d3f5,Aleff Ramon da Silva Costa
1,598e15296d8650eb27d52e3d,2017.2,LÓGICA DE PROGRAMAÇÃO - Turma 01A,20170039453,5991cb910be9d42f2f68b906,Amanda


Dias de início de cada semestre em datetime. Entre 2017.2 e 2019.2

In [0]:
d1 = pd.to_datetime("2017/07/24")
d2 = pd.to_datetime("2018/02/19")
d3 = pd.to_datetime("2018/07/30")
d4 = pd.to_datetime("2019/02/11")
d5 = pd.to_datetime("2019/07/22")

Essa função cria as colunas de semanas, conta a quantidade de submissões por semana, substitui os nan por 0 e por fim junta as tabelas

In [0]:
def trata(sub,table,turmas,d):
  t = pd.Timedelta("7 days")
  table = pd.DataFrame(columns=["idAluno","anoSubmissao"])
  table = criaColunaSemanas(table)
  table = buscaSubmissoesEntreSemanas(sub,d,t,table)
  table = somaConcatena(turmas, table) 
  return table  

Função que cria as colunas das semanas

In [0]:
def criaColunaSemanas(table):
  for i in range(1,22):
    table["semana "+str(i)] = 0
  return table

Busca as submissões entre as semanas

In [0]:
def buscaSubmissoesEntreSemanas(sub,d,t,table):
    for i in range(1,22):
      a = sub.loc[(sub["data"] >= d) & (sub["data"] < (d+t)),:]  
      b = a.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
      b.rename(columns={"questao":"semana "+str(i)},inplace=True)
      table = table.append(b,sort=False,ignore_index=True)
      d+=t
    return table

Função para deixar as submissões dos alunos em uma só linha e depois dar merge com o Outer Join, ou seja, mesmo que não tenham ligação, ele vai concatenar 

In [0]:
def somaConcatena(turmas,table):  
  turmas.replace(np.nan, 0, inplace=True)       
  table.replace(np.nan, 0, inplace=True)   
  table = table.groupby(["idAluno"]).sum()
  table = pd.merge(table,turmas,on="idAluno",how="outer")
  return table

Criação de DataFrames provisórios para cada semestre

In [0]:
prov1 = pd.DataFrame()
prov2 = pd.DataFrame()
prov3 = pd.DataFrame()
prov4 = pd.DataFrame()
prov5 = pd.DataFrame()

Chamada da função trata

In [0]:
prov1 = trata(sub,prov1,turmas,d1)
prov2 = trata(sub,prov2,turmas,d2)
prov3 = trata(sub,prov3,turmas,d3)
prov4 = trata(sub,prov4,turmas,d4)
prov5 = trata(sub,prov5,turmas,d5)

Inserindo os semestres de cada DataFrame correspondente

In [14]:
prov1.columns

Index(['idAluno', 'anoSubmissao', 'semana 1', 'semana 2', 'semana 3',
       'semana 4', 'semana 5', 'semana 6', 'semana 7', 'semana 8', 'semana 9',
       'semana 10', 'semana 11', 'semana 12', 'semana 13', 'semana 14',
       'semana 15', 'semana 16', 'semana 17', 'semana 18', 'semana 19',
       'semana 20', 'semana 21', 'idTurma', 'anoTurma', 'descricaoTurma',
       'matricula', 'nome'],
      dtype='object')

In [0]:
prov1["anoSubmissao"] = "2017.2"
prov2["anoSubmissao"] = "2018.1"
prov3["anoSubmissao"] = "2018.2"
prov4["anoSubmissao"] = "2019.1"
prov5["anoSubmissao"] = "2019.2"

In [0]:
def imprimaTamanho(tabela1,tabela2,tabela3,tabela4,tabela5):
  classes = [tabela1,tabela2,tabela3,tabela4,tabela5]
  for i in range(5):
    print("A quantidade de alunos é " + str(len(classes[i])) + " alunos")

Imprimindo o tamanho de cada DF

In [17]:
imprimaTamanho(prov1,prov2,prov3,prov4,prov5)

A quantidade de alunos é 1678 alunos
A quantidade de alunos é 1678 alunos
A quantidade de alunos é 1678 alunos
A quantidade de alunos é 1678 alunos
A quantidade de alunos é 1679 alunos


# Juntando com os 4 arquivos do Sigaa

In [0]:
arq1 = pd.read_csv("https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2017.2.csv")
arq2 = pd.read_csv("https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2018.1.csv")
arq3 = pd.read_csv("https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2018.2.csv")
arq4 = pd.read_csv("https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2019.1.csv")
arq5 = pd.read_csv("https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2019.2.csv")

Função que substitui NaN por 0

In [0]:
def nanPorZero(table):
  table.replace(np.nan, 0, inplace=True)
  return table

In [0]:
prov1 = nanPorZero(prov1)
prov2 = nanPorZero(prov2)
prov3 = nanPorZero(prov3)
prov4 = nanPorZero(prov4)
prov5 = nanPorZero(prov5)

In [0]:
arq1 = nanPorZero(arq1)
arq2 = nanPorZero(arq2)
arq3 = nanPorZero(arq3)
arq4 = nanPorZero(arq4)
arq5 = nanPorZero(arq5)

In [22]:
imprimaTamanho(prov1,prov2,prov3,prov4,prov5)
imprimaTamanho(arq1,arq2,arq3,arq4,arq5)

A quantidade de alunos é 1678 alunos
A quantidade de alunos é 1678 alunos
A quantidade de alunos é 1678 alunos
A quantidade de alunos é 1678 alunos
A quantidade de alunos é 1679 alunos
A quantidade de alunos é 432 alunos
A quantidade de alunos é 428 alunos
A quantidade de alunos é 461 alunos
A quantidade de alunos é 393 alunos
A quantidade de alunos é 449 alunos


Função para converter matricula para string

In [0]:
def transformaMatriculaParaString(table):
  table["matricula"] = table["matricula"].astype(str)
  return table

Transformando as matriculas do sigaa e das submissões para string

In [0]:
arq1 = transformaMatriculaParaString(arq1)
arq2 = transformaMatriculaParaString(arq2)
arq3 = transformaMatriculaParaString(arq3)
arq4 = transformaMatriculaParaString(arq4)
arq5 = transformaMatriculaParaString(arq5)

In [0]:
prov1 = transformaMatriculaParaString(prov1)
prov2 = transformaMatriculaParaString(prov2)
prov3 = transformaMatriculaParaString(prov3)
prov4 = transformaMatriculaParaString(prov4)
prov5 = transformaMatriculaParaString(prov5)

Renomeando o ano para criar uma destinção dos outros anos

In [0]:
arq1.rename(columns={"ano":"anoSigaa"},inplace=True)
arq2.rename(columns={"ano":"anoSigaa"},inplace=True)
arq3.rename(columns={"ano":"anoSigaa"},inplace=True)
arq4.rename(columns={"ano":"anoSigaa"},inplace=True)
arq5.rename(columns={"ano":"anoSigaa"},inplace=True)

In [27]:
arq1.head(1)

,Unnamed: 0,matricula,situacao,anoSigaa,turma
0,0,20170031910,REPROVADO,2017.2,04B


In [28]:
prov1.head(1)

,idAluno,anoSubmissao,semana 1,semana 2,semana 3,semana 4,semana 5,semana 6,semana 7,semana 8,semana 9,semana 10,semana 11,semana 12,semana 13,semana 14,semana 15,semana 16,semana 17,semana 18,semana 19,semana 20,semana 21,idTurma,anoTurma,descricaoTurma,matricula,nome
0,598cd5453bc48946977dcf66,2017.2,0.0,0.0,0.0,6.0,13.0,7.0,4.0,3.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2013023070,Gabriel Tibúrcio


Drop no unnamed

In [0]:
arq1 = arq1.drop(columns=["Unnamed: 0"])
arq2 = arq2.drop(columns=["Unnamed: 0"])
arq3 = arq3.drop(columns=["Unnamed: 0"])
arq4 = arq4.drop(columns=["Unnamed: 0"])
arq5 = arq5.drop(columns=["Unnamed: 0"])

Merge da tabelas de por ano das submissões no LoP com a tabela vinda do Sigaa

In [30]:
table1 = pd.merge(prov1,arq1,on="matricula",how="outer")
table2 = pd.merge(prov2,arq2,on="matricula",how="outer")
table3 = pd.merge(prov3,arq3,on="matricula",how="outer")
table4 = pd.merge(prov4,arq4,on="matricula",how="outer")
table5 = pd.merge(prov5,arq5,on="matricula",how="outer")
table5.head(2)

,idAluno,anoSubmissao,semana 1,semana 2,semana 3,semana 4,semana 5,semana 6,semana 7,semana 8,semana 9,semana 10,semana 11,semana 12,semana 13,semana 14,semana 15,semana 16,semana 17,semana 18,semana 19,semana 20,semana 21,idTurma,anoTurma,descricaoTurma,matricula,nome,situacao,anoSigaa,turma
0,598cd8b823daeb4b50a4b254,2019.2,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,1943220,Orivaldo,NaN,NaN,NaN
1,599246d441a16924126c3f31,2019.2,0.0,0.0,0.0,0.0,7.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,1980,Aquiles,NaN,NaN,NaN


In [31]:
imprimaTamanho(table1,table2,table3,table4,table5)

A quantidade de alunos é 1833 alunos
A quantidade de alunos é 1851 alunos
A quantidade de alunos é 1839 alunos
A quantidade de alunos é 1722 alunos
A quantidade de alunos é 1721 alunos


Substituindo NaN por 0

In [0]:
table1 = nanPorZero(table1)
table1 = nanPorZero(table1)
table1 = nanPorZero(table1)
table1 = nanPorZero(table1)
table1 = nanPorZero(table1)

Passando o ano submissão para flot para deixar igual aos outros anos

In [0]:
table1["anoSubmissao"] = table1["anoSubmissao"].astype(float)
table2["anoSubmissao"] = table2["anoSubmissao"].astype(float)
table3["anoSubmissao"] = table3["anoSubmissao"].astype(float)
table4["anoSubmissao"] = table4["anoSubmissao"].astype(float)
table5["anoSubmissao"] = table5["anoSubmissao"].astype(float)

Passando o campo turma para string

In [0]:
table1["turma"] = table1["turma"].astype(str)
table2["turma"] = table2["turma"].astype(str)
table3["turma"] = table3["turma"].astype(str)
table4["turma"] = table4["turma"].astype(str)
table5["turma"] = table5["turma"].astype(str)

Pegando os dados onde os anos do sigaa, turmas e submissão são iguais

In [0]:
def anosIguais(table):
  table = table.loc[(table["anoSigaa"]==table["anoTurma"]) & (table["anoSubmissao"]==table["anoSigaa"]),:]
  return table  

In [0]:
table1 = anosIguais(table1)
table2 = anosIguais(table2)
table3 = anosIguais(table3)
table4 = anosIguais(table4)
table5 = anosIguais(table5)

In [37]:
imprimaTamanho(table1,table2,table3,table4,table5)

A quantidade de alunos é 261 alunos
A quantidade de alunos é 212 alunos
A quantidade de alunos é 248 alunos
A quantidade de alunos é 227 alunos
A quantidade de alunos é 221 alunos


Implementação de funções

Função para imprimir a quantidade de alunos por ano

In [0]:
def quantidadePorAno(table):
  print("A quantidade total de alunos no semestre " + str(table["anoSigaa"].iloc[0]) + " são de " + str(table["idAluno"].count()) + " alunos")
  print("")

Alunos que estavam matriculados em uma turma e nunca logaram no sistema

In [0]:
def nuncaLogaram(table):
  b = table.loc[table["idAluno"]==0,:]
  print("A quantidade de alunos que estão matriculados em " + str(b["anoSigaa"].iloc[0]) + " e que não logaram são de " + str(b["idAluno"].count()))
  print("")
  print("As situações finais desses alunos são: ")
  print(b["situacao"].unique())

Alunos que nunca submeteram mas estão no sistema LoP

In [0]:
def nuncaSubmeteram(table):
  tam = table.iloc[:,2:23]
  table["somas"] = tam.sum(axis=1)
  print("")
  print("A quantidade de alunos que nunca submeteram em " + str (table["anoSigaa"].iloc[0]) + " são de " + str(table.loc[table["somas"]==0,"idAluno"].count()) + " alunos")
  print("")
  print("------------------------------------------------------------------------------")
  print("")

Função para chamar as outras funções

In [0]:
def chamaFuncoes(table):
  quantidadePorAno(table)
  nuncaLogaram(table)
  nuncaSubmeteram(table)

In [43]:
chamaFuncoes(table1)
chamaFuncoes(table2)
chamaFuncoes(table3)
chamaFuncoes(table4)
chamaFuncoes(table5)

A quantidade total de alunos no semestre 2017.2 são de 261 alunos

A quantidade de alunos que estão matriculados em 2017.2 e que não logaram são de 14

As situações finais desses alunos são: 
['REPROVADO POR MÉDIA E POR FALTAS' 'REPROVADO']

A quantidade de alunos que nunca submeteram em 2017.2 são de 46 alunos

------------------------------------------------------------------------------

A quantidade total de alunos no semestre 2018.1 são de 212 alunos

A quantidade de alunos que estão matriculados em 2018.1 e que não logaram são de 15

As situações finais desses alunos são: 
['REPROVADO POR MÉDIA E POR FALTAS' 'REPROVADO' 'APROVADO'
 'APROVADO POR NOTA']

A quantidade de alunos que nunca submeteram em 2018.1 são de 28 alunos

------------------------------------------------------------------------------

A quantidade total de alunos no semestre 2018.2 são de 248 alunos

A quantidade de alunos que estão matriculados em 2018.2 e que não logaram são de 8

As situações finais desses al

In [0]:
def matriculaDosAlunosQueNuncaLogaram(table):
  b = table.loc[table["idAluno"]==0,:]
  print("Os alunos que nunca logaram no semestre " + str(table["anoSigaa"].iloc[0]) + " são:")
  print(str(b["matricula"]))

In [0]:
def nomeDosAlunosQueNuncaSubmeteram(table):
  tam = table.iloc[:,2:23]
  table["somas"] = tam.sum(axis=1)
  print("Alunos que estão no sistema mas nunca submeteram em " + str (table["anoSigaa"].iloc[0]))
  print(str(table.loc[table["somas"]==0,["nome","matricula","turma"]]))

In [46]:
matriculaDosAlunosQueNuncaLogaram(table5)

Os alunos que nunca logaram no semestre 2019.2 são:
501    20180033954
505    20180136399
510    20180030415
512    20180041295
513    20180093550
514    20180042659
515    20180136684
522    20190073318
523    20190070352
524    20190080930
525    20190078978
526    20190075831
527    20190074914
528    20180040313
529    20180135444
530    20190079699
531    20190078154
532    20180039072
533    20180109589
Name: matricula, dtype: object


In [47]:
nomeDosAlunosQueNuncaSubmeteram(table5)

Alunos que estão no sistema mas nunca submeteram em 2019.2
                                   nome    matricula turma
501                                   0  20180033954   02C
505                                   0  20180136399   02B
510                                   0  20180030415   03C
512                                   0  20180041295   03B
513                                   0  20180093550   03C
514                                   0  20180042659   03D
515                                   0  20180136684   03D
522                                   0  20190073318   01D
523                                   0  20190070352   02A
524                                   0  20190080930   02A
525                                   0  20190078978   02A
526                                   0  20190075831   02B
527                                   0  20190074914   02B
528                                   0  20180040313   02B
529                                   0  20180135444   0

Concatenando tabelas

In [48]:
tabfinal = pd.concat([table1,table2,table3,table4], ignore_index=True)
tabfinal = tabfinal.drop(columns={"idAluno","matricula","nome"})
tabfinal.head()

,anoSubmissao,semana 1,semana 2,semana 3,semana 4,semana 5,semana 6,semana 7,semana 8,semana 9,semana 10,semana 11,semana 12,semana 13,semana 14,semana 15,semana 16,semana 17,semana 18,semana 19,semana 20,semana 21,idTurma,anoTurma,descricaoTurma,situacao,anoSigaa,turma,somas
0,2017.2,0.0,0.0,0.0,5.0,12.0,5.0,0.0,15.0,10.0,0.0,0.0,16.0,3.0,0.0,0.0,20.0,0.0,7.0,0.0,0.0,0.0,598e15296d8650eb27d52e3d,2017.2,LÓGICA DE PROGRAMAÇÃO - Turma 01A,APROVADO,2017.2,01A,93.0
1,2017.2,0.0,0.0,0.0,5.0,0.0,6.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,598e15296d8650eb27d52e5f,2017.2,LÓGICA DE PROGRAMAÇÃO - Turma 01B,APROVADO POR NOTA,2017.2,01B,18.0
2,2017.2,0.0,0.0,0.0,5.0,0.0,5.0,0.0,14.0,2.0,0.0,0.0,4.0,0.0,5.0,6.0,2.0,1.0,0.0,0.0,0.0,0.0,598e15296d8650eb27d52e5f,2017.2,LÓGICA DE PROGRAMAÇÃO - Turma 01B,APROVADO,2017.2,01B,44.0
3,2017.2,0.0,0.0,0.0,3.0,0.0,6.0,0.0,8.0,3.0,0.0,0.0,5.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,598e15296d8650eb27d52e5f,2017.2,LÓGICA DE PROGRAMAÇÃO - Turma 01B,APROVADO,2017.2,01B,31.0
4,2017.2,0.0,0.0,0.0,3.0,0.0,6.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,598e15296d8650eb27d52e5f,2017.2,LÓGICA DE PROGRAMAÇÃO - Turma 01B,APROVADO POR NOTA,2017.2,01B,16.0


In [0]:
tabfinal.to_csv(r"table.csv")

# Analisar os lixos

Excluindo as semanas

In [0]:
tb1 = table1.iloc[:,[0,1,-1,-2,-3,-4,-5,-6,-7]]
tb2 = table2.iloc[:,[0,1,-1,-2,-3,-4,-5,-6,-7]]
tb3 = table3.iloc[:,[0,1,-1,-2,-3,-4,-5,-6,-7]]
tb4 = table4.iloc[:,[0,1,-1,-2,-3,-4,-5,-6,-7]]
tb1.head(2)

In [0]:
print(len(tb1))
print(len(tb2))
print(len(tb3))
print(len(tb4))

In [0]:
tb1.replace(np.nan, 0, inplace=True)   
tb2.replace(np.nan, 0, inplace=True)   
tb3.replace(np.nan, 0, inplace=True)   
tb4.replace(np.nan, 0, inplace=True)   

In [0]:
tb1.anoSubmissao = tb1.anoSubmissao.astype(float)
tb2.anoSubmissao = tb2.anoSubmissao.astype(float)
tb3.anoSubmissao = tb3.anoSubmissao.astype(float)
tb4.anoSubmissao = tb4.anoSubmissao.astype(float)

Ano turma != ano sigaa então exclua

In [0]:
tb1 = tb1.loc[(tb1["anoSigaa"]==tb1["anoTurma"]),:]   # & (tb1["anoSubmissao"]==tb1["anoSigaa"]),:]
#tb2 = tb2.loc[(tb2["anoSigaa"]==tb2["anoTurma"]) & (tb2["anoSubmissao"]==tb2["anoSigaa"]),:]
#tb3 = tb3.loc[(tb3["anoSigaa"]==tb3["anoTurma"]) & (tb3["anoSubmissao"]==tb3["anoSigaa"]),:]
#tb4 = tb4.loc[(tb4["anoSigaa"]==tb4["anoTurma"]) & (tb4["anoSubmissao"]==tb4["anoSigaa"]),:]
tb1.head()

In [0]:
b = tb1.loc[(~tb1["turma"].str.contains("04")) & (~tb1["turma"].str.contains("05")),:]
b

In [0]:
a = tb1.loc[tb1["turma"]=="03A",:]

a.sort_values(by="matricula")

# Resto

Função para os 4 semestres

In [0]:
def trata(sub,prov1,prov2,prov3,prov4,prov5,prov6,prov7,prov8,prov9,d):
  
  #Criando os intervalos de semanas a partir da data de início do semestre
  t = pd.Timedelta("7 days")
  d11 = d+t
  d12 = d11+t
  d13 = d12+t
  d14 = d13+t
  d15 = d14+t
  d16 = d15+t
  d17 = d16+t
  d18 = d17+t
  d19 = d18+t
  d20 = d19+t
  d21 = d20+t
  

  #Buscando submissões entre os intervalos
  v11 = sub.loc[(sub["data"]>d11) & (sub["data"]<=d12),:]
  v12 = sub.loc[(sub["data"]>d12) & (sub["data"]<=d13),:]
  v13 = sub.loc[(sub["data"]>d13) & (sub["data"]<=d14),:]
  v14 = sub.loc[(sub["data"]>d14) & (sub["data"]<=d15),:]
  v15 = sub.loc[(sub["data"]>d15) & (sub["data"]<=d16),:]
  v16 = sub.loc[(sub["data"]>d16) & (sub["data"]<=d17),:]
  v17 = sub.loc[(sub["data"]>d17) & (sub["data"]<=d18),:]
  v18 = sub.loc[(sub["data"]>d18) & (sub["data"]<=d19),:]
  
  #Contando a quantidade de submissões dos alunos por semana
  s11 = v11.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  s11.rename(columns={"questao":"qsemana1"},inplace=True)
  s12 = v12.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  s12.rename(columns={"questao":"qsemana2"},inplace=True)
  s13 = v13.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  s13.rename(columns={"questao":"qsemana3"},inplace=True)
  s14 = v14.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  s14.rename(columns={"questao":"qsemana4"},inplace=True)
  s15 = v15.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  s15.rename(columns={"questao":"qsemana5"},inplace=True)
  s16 = v16.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  s16.rename(columns={"questao":"qsemana6"},inplace=True)
  s17 = v17.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  s17.rename(columns={"questao":"qsemana7"},inplace=True)
  s18 = v18.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  s18.rename(columns={"questao":"qsemana8"},inplace=True)
  
  #concatenação das tabelas
  prov1 = prov1.append(s11,sort=False,ignore_index=True)
  prov1 = prov1.append(s12,sort=False,ignore_index=True)
  prov1 = prov1.append(s13,sort=False,ignore_index=True)
  prov1 = prov1.append(s14,sort=False,ignore_index=True)
  prov1 = prov1.append(s15,sort=False,ignore_index=True)
  prov1 = prov1.append(s16,sort=False,ignore_index=True)
  prov1 = prov1.append(s17,sort=False,ignore_index=True)
  prov1 = prov1.append(s18,sort=False,ignore_index=True)
 
  #Quantidade de dias diferentes que houve submissão
  q11 = v11.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  q11 = q11.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  q11.rename(columns={"dia":"submeteu1"},inplace=True)
  q12 = v12.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  q12 = q12.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  q12.rename(columns={"dia":"submeteu2"},inplace=True)
  q13 = v13.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  q13 = q13.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  q13.rename(columns={"dia":"submeteu3"},inplace=True)
  q14 = v14.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  q14 = q14.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  q14.rename(columns={"dia":"submeteu4"},inplace=True)
  q15 = v15.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  q15 = q15.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  q15.rename(columns={"dia":"submeteu5"},inplace=True)
  q16 = v16.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  q16 = q16.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  q16.rename(columns={"dia":"submeteu6"},inplace=True)
  q17 = v17.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  q17 = q17.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  q17.rename(columns={"dia":"submeteu7"},inplace=True)
  q18 = v18.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  q18 = q18.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  q18.rename(columns={"dia":"submeteu8"},inplace=True)
  
  #concatenação das tabelas  
  prov2 = prov2.append(q11,sort=False,ignore_index=True)
  prov2 = prov2.append(q12,sort=False,ignore_index=True)
  prov2 = prov2.append(q13,sort=False,ignore_index=True)
  prov2 = prov2.append(q14,sort=False,ignore_index=True)
  prov2 = prov2.append(q15,sort=False,ignore_index=True)
  prov2 = prov2.append(q16,sort=False,ignore_index=True)
  prov2 = prov2.append(q17,sort=False,ignore_index=True)
  prov2 = prov2.append(q18,sort=False,ignore_index=True)
    
  #Verificando os turnos e criando um label, em seguida agrupando
  vf=sub.loc[(sub["data"]>d11) & (sub["data"]<=d19),:]
  sub0 = vf.loc[sub["turno"]==0,:]
  sub1 = vf.loc[sub["turno"]==1,:]
  sub2 = vf.loc[sub["turno"]==2,:]
  sub3 = vf.loc[sub["turno"]==3,:]
  sub0 = sub0.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  sub1 = sub1.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  sub2 = sub2.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  sub3 = sub3.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  sub0.rename(columns={"questao":"madrugada"},inplace=True)
  sub1.rename(columns={"questao":"manha"},inplace=True)
  sub2.rename(columns={"questao":"tarde"},inplace=True)
  sub3.rename(columns={"questao":"noite"},inplace=True)  
  
  #concatenação das tabelas  
  prov3 = prov3.append(sub0,sort=False,ignore_index=True)
  prov3 = prov3.append(sub1,sort=False,ignore_index=True)
  prov3 = prov3.append(sub2,sort=False,ignore_index=True)
  prov3 = prov3.append(sub3,sort=False,ignore_index=True)
   
  #Separando as semanas entre intervalos para os 5 labs
  s1 = sub.loc[(sub["data"]>d11) & (sub["data"]<=d13) & (sub["titulo"].str.contains("Lab")),:]
  s2 = sub.loc[(sub["data"]>d13) & (sub["data"]<=d15) & (sub["titulo"].str.contains("Lab")),:]
  s3 = sub.loc[(sub["data"]>d15) & (sub["data"]<=d17) & (sub["titulo"].str.contains("Lab")),:]
  s4 = sub.loc[(sub["data"]>d17) & (sub["data"]<=d19) & (sub["titulo"].str.contains("Lab")),:]
  s5 = sub.loc[(sub["data"]>d19) & (sub["data"]<=d21) & (sub["titulo"].str.contains("Lab")),:]
  
  #Separando as semanas entre intervalos para contar os exercicios submetidos
  h1 = sub.loc[(sub["data"]>d11) & (sub["data"]<=d13) & (sub["titulo"].str.contains("Exer")),:]
  h2 = sub.loc[(sub["data"]>d13) & (sub["data"]<=d15) & (sub["titulo"].str.contains("Exer")),:]
  h3 = sub.loc[(sub["data"]>d15) & (sub["data"]<=d17) & (sub["titulo"].str.contains("Exer")),:]
  h4 = sub.loc[(sub["data"]>d17) & (sub["data"]<=d19) & (sub["titulo"].str.contains("Exer")),:]
  h5 = sub.loc[(sub["data"]>d19) & (sub["data"]<=d21) & (sub["titulo"].str.contains("Exer")),:]
  
  #Contando as submissões dos alunos nas listas do lab a cada 2 semanas 
  ns1 = s1.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  ns1.rename(columns={"questao":"subListaLab23"},inplace=True)
  ns2 = s2.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  ns2.rename(columns={"questao":"subListaLab45"},inplace=True)
  ns3 = s3.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  ns3.rename(columns={"questao":"subListaLab67"},inplace=True)
  ns4 = s4.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  ns4.rename(columns={"questao":"subListaLab89"},inplace=True)
  ns5 = s5.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  ns5.rename(columns={"questao":"subListaLab1011"},inplace=True)
  
  #Contando as submissões dos alunos nas listas do lab a cada 2 semanas 
  ms1 = h1.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  ms1.rename(columns={"questao":"subListaExer23"},inplace=True)
  ms2 = h2.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  ms2.rename(columns={"questao":"subListaExer45"},inplace=True)
  ms3 = h3.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  ms3.rename(columns={"questao":"subListaExer67"},inplace=True)
  ms4 = h4.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  ms4.rename(columns={"questao":"subListaExer89"},inplace=True)
  ms5 = h5.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  ms5.rename(columns={"questao":"subListaExer1011"},inplace=True)
  
  #concatenação das tabelas
  prov4 = prov4.append(ns1,sort=False,ignore_index=True)
  prov4 = prov4.append(ns2,sort=False,ignore_index=True)
  prov4 = prov4.append(ns3,sort=False,ignore_index=True)
  prov4 = prov4.append(ns4,sort=False,ignore_index=True)
  prov4 = prov4.append(ns5,sort=False,ignore_index=True)
  
  #concatenação das tabelas
  prov7 = prov7.append(ms1,sort=False,ignore_index=True)
  prov7 = prov7.append(ms2,sort=False,ignore_index=True)
  prov7 = prov7.append(ms3,sort=False,ignore_index=True)
  prov7 = prov7.append(ms4,sort=False,ignore_index=True)
  prov7 = prov7.append(ms5,sort=False,ignore_index=True)
    
  #Agrupando por dia para saber a quantidade de questões submetidas de lab pelo aluno por semana
  qs1 = s1.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  qs1 = qs1.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  qs1.rename(columns={"dia":"subDistintasLab23"},inplace=True)
  qs2 = s2.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  qs2 = qs2.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  qs2.rename(columns={"dia":"subDistintasLab45"},inplace=True)
  qs3 = s3.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  qs3 = qs3.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  qs3.rename(columns={"dia":"subDistintasLab67"},inplace=True)
  qs4 = s4.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  qs4 = qs4.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  qs4.rename(columns={"dia":"subDistintasLab89"},inplace=True)  
  qs5 = s5.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  qs5 = qs5.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  qs5.rename(columns={"dia":"subDistintasLab1011"},inplace=True)
  
  #Agrupando por dia para saber a quantidade de questões submetidas de exercicio pelo aluno por semana
  hs1 = h1.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  hs1 = hs1.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  hs1.rename(columns={"dia":"subDistintasExer23"},inplace=True)
  hs2 = h2.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  hs2 = hs2.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  hs2.rename(columns={"dia":"subDistintasExer45"},inplace=True)
  hs3 = h3.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  hs3 = hs3.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  hs3.rename(columns={"dia":"subDistintasExer67"},inplace=True)
  hs4 = h4.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  hs4 = hs4.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  hs4.rename(columns={"dia":"subDistintasExer89"},inplace=True)  
  hs5 = h5.groupby(["idAluno","dia"])["questao"].count().reset_index().sort_values(by="idAluno")
  hs5 = hs5.groupby(["idAluno"])["dia"].count().reset_index().sort_values(by="idAluno")
  hs5.rename(columns={"dia":"subDistintasExer1011"},inplace=True)
  
  #concatenação das tabelas
  prov5 = prov5.append(qs1,sort=False,ignore_index=True)
  prov5 = prov5.append(qs2,sort=False,ignore_index=True)
  prov5 = prov5.append(qs3,sort=False,ignore_index=True)
  prov5 = prov5.append(qs4,sort=False,ignore_index=True)
  prov5 = prov5.append(qs5,sort=False,ignore_index=True)
  
  #concatenação das tabelas
  prov8 = prov8.append(hs1,sort=False,ignore_index=True)
  prov8 = prov8.append(hs2,sort=False,ignore_index=True)
  prov8 = prov8.append(hs3,sort=False,ignore_index=True)
  prov8 = prov8.append(hs4,sort=False,ignore_index=True)
  prov8 = prov8.append(hs5,sort=False,ignore_index=True)
  
  #Contando a quantidade de questoes de Lab diferentes por intervalo de semana
  m1 = s1.groupby(["idAluno","questao"])["titulo"].count().reset_index().sort_values(by="idAluno")
  m1 = m1.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  m1.rename(columns={"questao":"diferentesLabSemanas23"},inplace=True)
  
  m2 = s2.groupby(["idAluno","questao"])["titulo"].count().reset_index().sort_values(by="idAluno")
  m2 = m2.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  m2.rename(columns={"questao":"diferentesLabSemanas45"},inplace=True)
  m3 = s3.groupby(["idAluno","questao"])["titulo"].count().reset_index().sort_values(by="idAluno")
  m3 = m3.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  m3.rename(columns={"questao":"diferentesLabSemanas67"},inplace=True)
  m4 = s4.groupby(["idAluno","questao"])["titulo"].count().reset_index().sort_values(by="idAluno")
  m4 = m4.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  m4.rename(columns={"questao":"diferentesLabSemanas89"},inplace=True)
  m5 = s5.groupby(["idAluno","questao"])["titulo"].count().reset_index().sort_values(by="idAluno")
  m5 = m5.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  m5.rename(columns={"questao":"diferentesLabSemanas1011"},inplace=True)
  
  #Contando a quantidade de questoes de Exercicio diferentes por intervalo de semana
  p1 = h1.groupby(["idAluno","questao"])["titulo"].count().reset_index().sort_values(by="idAluno")
  p1 = p1.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  p1.rename(columns={"questao":"diferentesExerSemanas23"},inplace=True)
  p2 = h2.groupby(["idAluno","questao"])["titulo"].count().reset_index().sort_values(by="idAluno")
  p2 = p2.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  p2.rename(columns={"questao":"diferentesExerSemanas45"},inplace=True)
  p3 = h3.groupby(["idAluno","questao"])["titulo"].count().reset_index().sort_values(by="idAluno")
  p3 = p3.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  p3.rename(columns={"questao":"diferentesExerSemanas67"},inplace=True)
  p4 = h4.groupby(["idAluno","questao"])["titulo"].count().reset_index().sort_values(by="idAluno")
  p4 = p4.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  p4.rename(columns={"questao":"diferentesExerSemanas89"},inplace=True)
  p5 = h5.groupby(["idAluno","questao"])["titulo"].count().reset_index().sort_values(by="idAluno")
  p5 = p5.groupby(["idAluno"])["questao"].count().reset_index().sort_values(by="idAluno")
  p5.rename(columns={"questao":"diferentesExerSemanas1011"},inplace=True) 

  #concatenação das tabelas  
  prov6 = prov6.append(m1,sort=False,ignore_index=True)
  prov6 = prov6.append(m2,sort=False,ignore_index=True)
  prov6 = prov6.append(m3,sort=False,ignore_index=True)
  prov6 = prov6.append(m4,sort=False,ignore_index=True)
  prov6 = prov6.append(m5,sort=False,ignore_index=True)

  #concatenação das tabelas
  prov9 = prov9.append(p1,sort=False,ignore_index=True)
  prov9 = prov9.append(p2,sort=False,ignore_index=True)
  prov9 = prov9.append(p3,sort=False,ignore_index=True)
  prov9 = prov9.append(p4,sort=False,ignore_index=True)
  prov9 = prov9.append(p5,sort=False,ignore_index=True)

  
  return prov1,prov2,prov3,prov4,prov5,prov6,prov7,prov8,prov9

Criando um dataframe provisório para utilizar com o append

In [0]:
prov1 = pd.DataFrame()
prov2 = pd.DataFrame()
prov3 = pd.DataFrame()
prov4 = pd.DataFrame()
prov5 = pd.DataFrame()
prov6 = pd.DataFrame()
prov7 = pd.DataFrame()
prov8 = pd.DataFrame()
prov9 = pd.DataFrame()

prov1["idAluno"]=0
prov1["qsemana1"]=0
prov1["qsemana2"]=0
prov1["qsemana3"]=0
prov1["qsemana4"]=0
prov1["qsemana5"]=0
prov1["qsemana6"]=0
prov1["qsemana7"]=0
prov1["qsemana8"]=0

prov2["idAluno"]=0
prov2["submeteu1"]=0
prov2["submeteu2"]=0
prov2["submeteu3"]=0
prov2["submeteu4"]=0
prov2["submeteu5"]=0
prov2["submeteu6"]=0
prov2["submeteu7"]=0
prov2["submeteu8"]=0

prov3["idAluno"]=0
prov3["madrugada"]=0
prov3["manha"]=0
prov3["tarde"]=0
prov3["noite"]=0

prov4["idAluno"]=0
prov4["subListaLab23"]=0
prov4["subListaLab45"]=0
prov4["subListaLab67"]=0
prov4["subListaLab89"]=0
prov4["subListaLab1011"]=0

prov5["idAluno"]=0
prov5["subDistintasLab23"]=0
prov5["subDistintasLab45"]=0
prov5["subDistintasLab67"]=0
prov5["subDistintasLab89"]=0
prov5["subDistintasLab1011"]=0

prov6["idAluno"]=0
prov6["diferentesLabSemanas23"]=0
prov6["diferentesLabSemanas45"]=0
prov6["diferentesLabSemanas67"]=0
prov6["diferentesLabSemanas89"]=0
prov6["diferentesLabSemanas1011"]=0

#mesmo processo agora para listas de exercicios
prov7["idAluno"]=0
prov7["subListaExer23"]=0
prov7["subListaExer45"]=0
prov7["subListaExer67"]=0
prov7["subListaExer89"]=0
prov7["subListaExer1011"]=0

prov8["idAluno"]=0
prov8["subDistintasExer23"]=0
prov8["subDistintasExer45"]=0
prov8["subDistintasExer67"]=0
prov8["subDistintasExer89"]=0
prov8["subDistintasExer1011"]=0

prov9["idAluno"]=0
prov9["diferentesExerSemanas23"]=0
prov9["diferentesExerSemanas45"]=0
prov9["diferentesExerSemanas67"]=0
prov9["diferentesExerSemanas89"]=0
prov9["diferentesExerSemanas1011"]=0


Chamada de funções

In [0]:
prov1,prov2,prov3,prov4,prov5,prov6,prov7,prov8,prov9 = trata(sub,prov1,prov2,prov3,prov4,prov5,prov6,prov7,prov8,prov9,d1)
prov1,prov2,prov3,prov4,prov5,prov6,prov7,prov8,prov9 = trata(sub,prov1,prov2,prov3,prov4,prov5,prov6,prov7,prov8,prov9,d2)
prov1,prov2,prov3,prov4,prov5,prov6,prov7,prov8,prov9 = trata(sub,prov1,prov2,prov3,prov4,prov5,prov6,prov7,prov8,prov9,d3)
prov1,prov2,prov3,prov4,prov5,prov6,prov7,prov8,prov9 = trata(sub,prov1,prov2,prov3,prov4,prov5,prov6,prov7,prov8,prov9,d4)

In [0]:
prov9.head(4)

Limpando dados, depois agrupando para formar toda a tabela

In [0]:
prov1.replace(np.nan, 0, inplace=True)
prov2.replace(np.nan, 0, inplace=True)
prov3.replace(np.nan, 0, inplace=True)
prov4.replace(np.nan, 0, inplace=True)
prov5.replace(np.nan, 0, inplace=True)
prov6.replace(np.nan, 0, inplace=True)
prov7.replace(np.nan, 0, inplace=True)
prov8.replace(np.nan, 0, inplace=True)
prov9.replace(np.nan, 0, inplace=True)

prov1 = prov1.groupby(["idAluno"]).sum()
prov2 = prov2.groupby(["idAluno"]).sum()
prov3 = prov3.groupby(["idAluno"]).sum()
prov4 = prov4.groupby(["idAluno"]).sum()
prov5 = prov5.groupby(["idAluno"]).sum()
prov6 = prov6.groupby(["idAluno"]).sum()
prov7 = prov7.groupby(["idAluno"]).sum()
prov8 = prov8.groupby(["idAluno"]).sum()
prov9 = prov9.groupby(["idAluno"]).sum()

In [0]:
prov9.head(6)

Juntando a tabela de notas com as 3 novas tabelas

In [0]:
table1=pd.merge(pd.merge(nota,prov1,on="idAluno"),pd.merge(prov2,prov3,on="idAluno"),on="idAluno")
table2=pd.merge(pd.merge(table1,prov4,on="idAluno"),pd.merge(prov7,prov5,on="idAluno"),on="idAluno")
table=pd.merge(pd.merge(table2,prov8,on="idAluno"),pd.merge(prov6,prov9,on="idAluno"),on="idAluno")
table.head()

In [0]:
a=table.loc[table["idAluno"]=="59ad93041ec9d13cacb45e36",:]
a

In [0]:
table["idAluno"].value_counts()

Contando a quantidade de questões por Lista

In [0]:
qnt = pd.DataFrame()
sub = sub.loc[(sub["titulo"].str.contains("Lab")) | (sub["titulo"].str.contains("Exer")),:]
qnt = sub.groupby("idLista").count()
#qnt = sub.groupby(["idLista","questao"])["titulo"].count().reset_index()
#qnt.rename(columns={"questao":"quantidade"}, inplace=True)
qnt.head()

Porcentagem Máxima por aluno em cada questão da lista

In [0]:
maxi = pd.DataFrame()
maxi = sub.groupby(["idLista","idAluno","questao"]).max().reset_index()
maxi.rename(columns={"porcentagemAcerto":"porcentagemMaxima"}, inplace=True)
maxi.head(1)

Quantidade de questoes feitas por aluno em cada lista

In [0]:
quantidade = NotaMaxQuestao.groupby(["titulo","idAluno"]).count()
quantidade.rename(columns={"quantidade":"questoesFeitas"}, inplace=True)
quantidade= quantidade.reset_index()
quantidade= quantidade[["titulo","idAluno","questoesFeitas"]]
print(quantidade.columns)
quantidade.head(1)

**Nota final dos alunos por lista**

In [0]:
NotaLista = NotaMaxQuestao.groupby(["titulo","idAluno"]).sum()
NotaLista.rename(columns={"porcentagemMaxima":"SomaLista", "quantidade":"questoesFeitas"}, inplace=True)
NotaLista = NotaLista.drop('ano',axis=1)
NotaLista = NotaLista.reset_index()
NotaLista = NotaLista[["titulo",'idAluno','SomaLista']]
print(NotaLista.columns)
NotaLista.head(1)

In [0]:
table.describe()

In [0]:
database = table.iloc[:,:]
database = database.drop(columns={"matricula","idAluno","idTurma"})
database.head(1)

Gerando .csv

In [0]:
database.to_csv(r"DataBaseLop.csv")


# Glossario

*  qsemana == quantidade de submissões dos alunos por semana 
---


*  submeteu = quantidade de dias diferentes que houve submissão
---


*  subListaLab  == quantidade de submissões na lista de lab a cada 2 semanas
*  subListaExer  == quantidade de submissões na lista de exer a cada 2 semanas


---


*  subDistintasLab  == quantidade de dias em que submeteu a cada 2 semanas nas listas de lab
*  subDistintasExer  == quantidade de dias em que submeteu a cada 2 semanas nas listas de exer


---


*  diferentesLabSemanas  == quantidade de questões diferentes submetidas nas listas de lab a cada 2 semanas
*  diferentesExerSemanas  == quantidade de questões diferentes submetidas nas listas de lab a cada 2 semanas


---

*  situacao == indica se o aluno foi aprovado (1) ou reprovado (0)



---

*  qsub == quantidade de submissões das listas correspondente (seja lab ou exercicio)

---
*  qsubp == quantidade de questões submetidas para a prova 1 (L1,L2,L3) ou da prova 2 (L4,L5)


---
*  totalsub == total de submissões feitas pelo aluno


---

*  igualACem == quantidade de submissões em que o aluno acertou 100%, seja nas L123 ou L45


# Análise de dos melhores alunos de 18.2 e 19.1

In [0]:
df = table.loc[((table["ano"]==2018.2) | (table["ano"]==2019.1)) & (table["mediaProvas"] > 70),:]
df.describe()

In [0]:
df

# Rede SOM

SOM == todos os atributos numéricos para classificar, gerando uma matriz 10x10 da rede SOM

In [0]:
ds = table.loc[(table["ano"]==2018.2) | (table["ano"]==2019.1),:]
dataset = ds.iloc[:,44:74]
dataset = dataset.astype(np.float16)
dataset["situacao"] = table["situacao"]

In [0]:
dataset.head(2)

In [0]:
dataset2 = pd.DataFrame()
dataset2["situacao"] = dataset["situacao"]
dataset2["subListaLab23"] = dataset["subListaLab23"]
dataset2["subDistintasLab23"] = dataset["subDistintasLab23"]
dataset2["diferentesLabSemanas23"] = dataset["diferentesLabSemanas23"]
dataset2["subListaExer23"] = dataset["subListaExer23"]
dataset2["subDistintasExer23"] = dataset["subDistintasExer23"]
dataset2["diferentesExerSemanas23"] = dataset["diferentesExerSemanas23"]

In [0]:
dataset2.head()

In [0]:
dataset3 = table.iloc[:,54:57]
dataset3["situacao"] = table["situacao"]
dataset3.head()

In [0]:
dataset3.iloc[:,-1]

In [0]:
dataset2.shape

In [0]:
dataset.shape

In [0]:
X_train = dataset.iloc[:, 0:29].values 
target_train = dataset.iloc[:,30].values

[row, col] = X_train.shape
print (row," ",col)

print(X_train[1,:])

In [0]:
# Instalando a MiniSOM
!pip install minisom

In [0]:
# Training the SOM
tamanhoXdaRede = 10; 
tamanhoYdaRede = 10; 

quantidadeCaracteristicas = col
from minisom import MiniSom
som = MiniSom(x = tamanhoXdaRede, y = tamanhoYdaRede, input_len = quantidadeCaracteristicas, sigma = 1.0, learning_rate = 0.4)
som.pca_weights_init(X_train)

In [0]:
som.train_random(data = X_train, num_iteration = 40000)

In [0]:
# Obtem o vetor de pesos da rede treinada 
pesos = som.get_weights()

In [0]:
# Mostra um peso 
x = np.arange(quantidadeCaracteristicas)
plt.bar(x, pesos[1,2,:])
plt.show()

In [0]:
# obtendo os valores das repostas 
Y_train = target_train
Y_train[1]

In [0]:
# encontra o vencedor 
x = X_train[1,:]
pos = som.winner(x)


# matriz de zeros para contador de aprovados 
MContAp = np.zeros((tamanhoXdaRede,tamanhoYdaRede))
# matriz de zeros para o contador de reprovados 
MContT = np.zeros((tamanhoXdaRede,tamanhoYdaRede))
cont = 0; 
for x in X_train: 
  pos = som.winner(x)
  if (Y_train[cont] >= 1): #Aprovado 
    MContAp[pos] += 1
  MContT[pos] += 1
  cont= cont+1

In [0]:
print("Total:")
print(MContT)


print("Aprovados")
print(MContAp)

In [0]:
# Mostra todos os pesos 
cont = 1;
for row in pesos:
  for elem in row:
    plt.subplot(tamanhoXdaRede,tamanhoYdaRede,cont)
    cont=cont+1
    plt.axis([-1, 6, 0, 10])
    plt.bar(x, elem)
    plt.plot([-1,29],[1,1],'r')
plt.show()
#plt.savefig("test.jpg", dpi=250)

**Para dados reduzidos**

In [0]:
X_train = dataset3.iloc[:,:5].values 
target_train = dataset3.iloc[:,5:].values

[row, col] = X_train.shape
print (row," ",col)

print(X_train[1,:])

In [0]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
X = sc.fit_transform(X_train)

In [0]:
# Training the SOM
tamanhoXdaRede = 6; 
tamanhoYdaRede = 6; 

quantidadeCaracteristicas = col

In [0]:
from minisom import MiniSom
som = MiniSom(x = tamanhoXdaRede, y = tamanhoYdaRede, input_len = quantidadeCaracteristicas, sigma = 1.0, learning_rate = 0.4)
som.pca_weights_init(X_train)

In [0]:
som.train_random(data = X_train, num_iteration = 40000)

In [0]:
# Obtem o vetor de pesos da rede treinada 
pesos = som.get_weights()

In [0]:
# Mostra um peso 
x = np.arange(quantidadeCaracteristicas)
plt.bar(x, pesos[1,2,:])
plt.show()

In [0]:
# obtendo os valores das repostas 
Y_train = target_train
Y_train[1]

In [0]:
# encontra o vencedor 
x = X_train[1,:]
pos = som.winner(x)


# matriz de zeros para contador de aprovados 
MContAp = np.zeros((tamanhoXdaRede,tamanhoYdaRede))
# matriz de zeros para o contador de reprovados 
MContT = np.zeros((tamanhoXdaRede,tamanhoYdaRede))
cont = 0; 
for x in X_train: 
  pos = som.winner(x)
  if (Y_train[cont] >= 1): #Aprovado 
    MContAp[pos] += 1
  MContT[pos] += 1
  cont= cont+1

In [0]:
print("Total:")
print(MContT)


print("Aprovados")
print(MContAp)

In [0]:
# Mostra todos os pesos 
cont = 1;
for row in pesos:
  for elem in row:
    plt.subplot(tamanhoXdaRede,tamanhoYdaRede,cont)
    cont=cont+1
    plt.axis([-1, 6, 0, 10])
    plt.bar(x, elem)
    plt.plot([-1,8],[1,1],'r')
plt.show()
#plt.savefig("test.jpg", dpi=250)

In [0]:
c = np.divide(MContAp,MContT)
# Substitui nan por 0 
res = c
where_are_NaNs = np.isnan(res)

res[where_are_NaNs] = 0
print(res)

# Testes com a SOM

In [0]:
data = [[ 0.80, 0.55, 0.22, 0.03],
[ 0.82, 0.50, 0.23, 0.03],
[ 0.80, 0.54, 0.22, 0.03],
[ 0.80, 0.53, 0.26, 0.03],
[ 0.79, 0.56, 0.22, 0.03],
[ 0.75, 0.60, 0.25, 0.03],
[ 0.77, 0.59, 0.22, 0.03]]

In [0]:
from minisom import MiniSom

In [0]:
som = MiniSom(6, 6, 4, sigma=0.3, learning_rate=0.5)
som.train_random(data, 100)

In [0]:
qnt = som.quantization(data)

# SVM RBF - teste

SVC RBF ao longo das semanas 

sublistalab23
subListaExer23
subDistintasLab23
subDistintasExer23
diferentesLabSemanas23
diferentesExerSemanas23


In [0]:
from matplotlib.colors import Normalize
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

Função para mover um ponto médio de um mapa de cores em torno dos valores de interesse

In [0]:
class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

Selecionando as colunas de interesse

In [0]:
df = table.iloc[:,44:74]

In [0]:
y = table["situacao"]

In [0]:
df.head()

In [0]:
X = df.to_numpy()
y = y.to_numpy()

In [0]:
X_2d = X
X_2d = X_2d[y > 0]
y_2d = y
y_2d -= 1

In [0]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_2d = scaler.fit_transform(X_2d)

In [0]:
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(X, y)

In [0]:
print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

In [0]:
C_2d_range = [1e-2, 1, 1e2]
gamma_2d_range = [1e-1, 1, 1e1]
classifiers = []
for C in C_2d_range:
    for gamma in gamma_2d_range:
        clf = SVC(C=C, gamma=gamma)
        clf.fit(X_2d, y_2d)
        classifiers.append((C, gamma, clf))

In [0]:
plt.figure(figsize=(8, 6))

for (k, (C, gamma, clf)) in enumerate(classifiers):
    # evaluate decision function in a grid
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # visualize decision function for these parameters
    plt.subplot(len(C_2d_range), len(gamma_2d_range), k + 1)
    plt.title("gamma=10^%d, C=10^%d" % (np.log10(gamma), np.log10(C)),
              size='medium')

    # visualize parameter's effect on decision function
    plt.pcolormesh(xx, yy, -Z, cmap=plt.cm.RdBu)
    plt.scatter(X_2d[:, 0], X_2d[:, 1], c=y_2d, cmap=plt.cm.RdBu_r,
                edgecolors='k')
    plt.xticks(())
    plt.yticks(())
    plt.axis('tight')

scores = grid.cv_results_['mean_test_score'].reshape(len(C_range),
                                                     len(gamma_range))

In [0]:
plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot,
           norm=MidpointNormalize(vmin=0.2, midpoint=0.92))
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar()
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.title('Validation accuracy')
plt.show()

# SVM RBF


In [0]:
X = table.iloc[:,54:57].values
y = table["situacao"].values

In [0]:
X[0:3,:]

In [0]:
X.shape

In [0]:
y.shape

In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 10)

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
# Fitting SVM to the Training set
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

#classifier = SVC(kernel = 'linear', random_state = 0)
classifier = Pipeline((
("scaler", StandardScaler()),
("svm_clf", SVC(kernel="rbf", gamma=0.9, C=0.5))
))

classifier.fit(X_train, y_train)

In [0]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

print(y_test[0:35])
print(y_pred[0:35])

In [0]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

cm

In [0]:
# Visualising the Training set results
from matplotlib.colors import ListedColormap
X_set, y_set = X_train, y_train
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('blue', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('SVM (Training set)')
#plt.xlabel('Age')
#plt.ylabel('Estimated Salary')
plt.legend()
plt.show()

# Modelagem 2

In [0]:
NewTable = table[(table["ano"] != "2018.2") || (table["ano"] != "2019.1")]

# Clusterizando

In [0]:
from sklearn.cluster import KMeans

In [0]:
x = table.iloc[:,24:32]
x.head(1)

Escolhendo a quantidade de grupos e depois treinando

In [0]:
kmeans = KMeans(2)
kmeans.fit(x)

Indentificando os clusters

In [0]:
identified_clusters = kmeans.fit_predict(x)
identified_clusters

Inserindo na tabela

In [0]:
table_clusters = table.copy()
table_clusters["cluster"] = identified_clusters
table_clusters.head(2)

In [0]:
plt.scatter("ano","qsemana8",data=table_clusters,c=table_clusters["situacao"],cmap="rainbow")
#plt.xlim(-200,200)
#plt.ylim(-90,90)
plt.show()

In [0]:
plt.scatter("ano","qsemana8",data=table_clusters,c=table_clusters["cluster"],cmap="rainbow")
#plt.xlim(-200,200)
#plt.ylim(-90,90)
plt.show()

# Testes com machine learning


SVM com tabela clusterizada


In [0]:
dataset = table_clusters.drop(columns=["tituloProva2","tituloProva1","idAluno","ano","matricula","idTurma","igualACeml45","igualACeml123","qsub4","totalsub","qsubp2","qsubp1","qsub5","qsub3","qsub2","qsub1","mediaProvas","quantidadeProva2","questoesFeitasProva2","notaProva2","quantidadeProva1","questoesFeitasProva1","notaProva1","noite","tarde","manha","madrugada","submeteu8","submeteu7","submeteu6","submeteu5","submeteu4","submeteu3","submeteu2","submeteu1"])
#dataset = table_clusters.drop(columns=["tituloProva2","tituloProva1","idAluno","ano","matricula","idTurma","igualACeml45","igualACeml123","qsub4","totalsub","qsubp2","qsubp1","qsub5","qsub3","qsub2","qsub1","mediaProvas","quantidadeProva2","questoesFeitasProva2","notaProva2","quantidadeProva1","questoesFeitasProva1","notaProva1","submeteu8","submeteu7","submeteu6","submeteu5","submeteu4","submeteu3","submeteu2","submeteu1"])
dataset.head(1)

In [0]:
dataset = 

In [0]:
teste = table.groupby("ano")["situacao"].count()
teste


In [0]:
data = dataset
target = dataset["situacao"]
data.drop(columns=["situacao"],inplace=True)
data.head(2)

In [0]:
#import the necessary module
from sklearn.model_selection import train_test_split
#split data set into train and test sets
data_train, data_test, target_train, target_test = train_test_split(data,target, test_size = 0.30, random_state = 10)

In [0]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
#create an object of type LinearSVC
svc_model = LinearSVC(random_state=4)
#train the algorithm on training data and predict using the testing data
pred = svc_model.fit(data_train, target_train).predict(data_test)
#print the accuracy score of the model
print("LinearSVC accuracy : ",accuracy_score(target_test, pred, normalize = True))

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(svc_model, classes=["REPROVADO", "APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

# Naive Bayes, Regressão Linear e KNN com todos os dados

Documentação do algoritmo de Naive Bayes https://www.dataquest.io/blog/sci-kit-learn-tutorial/

Esse primeiro experimento conta com os dados de submissão por semana, o número de dias diferentes que ele submeteu e a quantidade de submissão por turno

In [0]:
datasetBayes=table
datasetBayes=datasetBayes.drop(columns=["tituloProva2","tituloProva1","idAluno","ano","matricula","idTurma","igualACeml45","igualACeml123","qsub4","totalsub","qsubp2","qsubp1","qsub5","qsub3","qsub2","qsub1","mediaProvas","quantidadeProva2","questoesFeitasProva2","notaProva2","quantidadeProva1","questoesFeitasProva1","notaProva1"])
datasetBayes.head(1)

In [0]:
#import the necessary module
from sklearn import preprocessing
# create the Labelencoder object
le = preprocessing.LabelEncoder()
#convert the categorical columns into numeric
datasetBayes["situacao"]                       = le.fit_transform(datasetBayes["situacao"])
datasetBayes["qsemana1"]                       = le.fit_transform(datasetBayes["qsemana1"])
datasetBayes["qsemana2"]                       = le.fit_transform(datasetBayes["qsemana2"])
datasetBayes["qsemana3"]                       = le.fit_transform(datasetBayes["qsemana3"])
datasetBayes["qsemana4"]                       = le.fit_transform(datasetBayes["qsemana4"])
datasetBayes["qsemana5"]                       = le.fit_transform(datasetBayes["qsemana5"])
datasetBayes["qsemana6"]                       = le.fit_transform(datasetBayes["qsemana6"])
datasetBayes["qsemana7"]                       = le.fit_transform(datasetBayes["qsemana7"])
datasetBayes["qsemana8"]                       = le.fit_transform(datasetBayes["qsemana8"])
datasetBayes["submeteu1"]                      = le.fit_transform(datasetBayes["submeteu1"])
datasetBayes["submeteu2"]                      = le.fit_transform(datasetBayes["submeteu2"])
datasetBayes["submeteu3"]                      = le.fit_transform(datasetBayes["submeteu3"])
datasetBayes["submeteu4"]                      = le.fit_transform(datasetBayes["submeteu4"])
datasetBayes["submeteu5"]                      = le.fit_transform(datasetBayes["submeteu5"])
datasetBayes["submeteu6"]                      = le.fit_transform(datasetBayes["submeteu6"])
datasetBayes["submeteu7"]                      = le.fit_transform(datasetBayes["submeteu7"])
datasetBayes["submeteu8"]                      = le.fit_transform(datasetBayes["submeteu8"])
datasetBayes["madrugada	"]                     = le.fit_transform(datasetBayes["madrugada"])
datasetBayes["manha"]                          = le.fit_transform(datasetBayes["manha"])
datasetBayes["tarde"]                          = le.fit_transform(datasetBayes["tarde"])
datasetBayes["noite"]                          = le.fit_transform(datasetBayes["noite"])

datasetBayes.head()

Retirando o campo Situação, porque é o que queremos prever

In [0]:
data = datasetBayes
target = datasetBayes["situacao"]
data.drop(columns=["situacao"],inplace=True)
data.head(2)

In [0]:
#import the necessary module
from sklearn.model_selection import train_test_split
#split data set into train and test sets
data_train, data_test, target_train, target_test = train_test_split(data,target, test_size = 0.30, random_state = 10)

In [0]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
#create an object of the type GaussianNB
gnb = GaussianNB()
#train the algorithm on training data and predict using the testing data
pred = gnb.fit(data_train, target_train).predict(data_test)
#print(pred.tolist())
#print the accuracy score of the model
print("Naive-Bayes accuracy : ",accuracy_score(target_test, pred, normalize = True))

In [0]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
#create an object of type LinearSVC
svc_model = LinearSVC(random_state=10)
#train the algorithm on training data and predict using the testing data
pred = svc_model.fit(data_train, target_train).predict(data_test)
#print the accuracy score of the model
print("LinearSVC accuracy : ",accuracy_score(target_test, pred, normalize = True))

In [0]:
#import necessary modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#create object of the lassifier
neigh = KNeighborsClassifier(n_neighbors=3)
#Train the algorithm
neigh.fit(data_train, target_train)
# predict the response
pred = neigh.predict(data_test)
# evaluate accuracy
print ("KNeighbors accuracy score : ",accuracy_score(target_test, pred))

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(gnb, classes=["REPROVADO", "APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(svc_model, classes=["REPROVADO", "APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(neigh, classes=["REPROVADO","APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

# Naive Bayes, Regressão Linear e KNN com submissões por semana

Este segundo experimento com os dados de submissões por semana para descobrir se o aluno foi ou não aprovado

In [0]:
datasetBayes=table
datasetBayes=datasetBayes.drop(columns=["tituloProva2","tituloProva1","idAluno","ano","matricula","idTurma","igualACeml45","igualACeml123","qsub4","totalsub","qsubp2","qsubp1","qsub5","qsub3","qsub2","qsub1","mediaProvas","quantidadeProva2","questoesFeitasProva2","notaProva2","quantidadeProva1","questoesFeitasProva1","notaProva1","noite","tarde","manha","madrugada","submeteu8","submeteu7","submeteu6","submeteu5","submeteu4","submeteu3","submeteu2","submeteu1"])
datasetBayes.head(1)

In [0]:
#import the necessary module
from sklearn import preprocessing
# create the Labelencoder object
le = preprocessing.LabelEncoder()
#convert the categorical columns into numeric
datasetBayes["situacao"]                       = le.fit_transform(datasetBayes["situacao"])
datasetBayes["qsemana1"]                       = le.fit_transform(datasetBayes["qsemana1"])
datasetBayes["qsemana2"]                       = le.fit_transform(datasetBayes["qsemana2"])
datasetBayes["qsemana3"]                       = le.fit_transform(datasetBayes["qsemana3"])
datasetBayes["qsemana4"]                       = le.fit_transform(datasetBayes["qsemana4"])
datasetBayes["qsemana5"]                       = le.fit_transform(datasetBayes["qsemana5"])
datasetBayes["qsemana6"]                       = le.fit_transform(datasetBayes["qsemana6"])
datasetBayes["qsemana7"]                       = le.fit_transform(datasetBayes["qsemana7"])
datasetBayes["qsemana8"]                       = le.fit_transform(datasetBayes["qsemana8"])

datasetBayes.head()

Retirando o campo Situação, porque é o que queremos prever

In [0]:
data = datasetBayes
target = datasetBayes["situacao"]
data.drop(columns=["situacao"],inplace=True)
data.head(2)

In [0]:
#import the necessary module
from sklearn.model_selection import train_test_split
#split data set into train and test sets
data_train, data_test, target_train, target_test = train_test_split(data,target, test_size = 0.30, random_state = 10)

In [0]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
#create an object of the type GaussianNB
gnb = GaussianNB()
#train the algorithm on training data and predict using the testing data
pred = gnb.fit(data_train, target_train).predict(data_test)
#print(pred.tolist())
#print the accuracy score of the model
print("Naive-Bayes accuracy : ",accuracy_score(target_test, pred, normalize = True))

In [0]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
#create an object of type LinearSVC
svc_model = LinearSVC(random_state=14)
#train the algorithm on training data and predict using the testing data
pred = svc_model.fit(data_train, target_train).predict(data_test)
#print the accuracy score of the model
print("LinearSVC accuracy : ",accuracy_score(target_test, pred, normalize = True))

In [0]:
#import necessary modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#create object of the lassifier
neigh = KNeighborsClassifier(n_neighbors=3)
#Train the algorithm
neigh.fit(data_train, target_train)
# predict the response
pred = neigh.predict(data_test)
# evaluate accuracy
print ("KNeighbors accuracy score : ",accuracy_score(target_test, pred))

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(gnb, classes=["REPROVADO", "APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(svc_model, classes=["REPROVADO", "APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(neigh, classes=["REPROVADO","APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data


# Naive Bayes, Regressão Linear e KNN com submissões por semana e por turno

Neste terceiro experimento foram usado dados da quantidade de submissão por semana e por turno

In [0]:
datasetBayes=table
datasetBayes=datasetBayes.drop(columns=["tituloProva2","tituloProva1","idAluno","ano","matricula","idTurma","igualACeml45","igualACeml123","qsub4","totalsub","qsubp2","qsubp1","qsub5","qsub3","qsub2","qsub1","mediaProvas","quantidadeProva2","questoesFeitasProva2","notaProva2","quantidadeProva1","questoesFeitasProva1","notaProva1","submeteu8","submeteu7","submeteu6","submeteu5","submeteu4","submeteu3","submeteu2","submeteu1"])
datasetBayes.head(1)

In [0]:
#import the necessary module
from sklearn import preprocessing
# create the Labelencoder object
le = preprocessing.LabelEncoder()
#convert the categorical columns into numeric
datasetBayes["situacao"]                       = le.fit_transform(datasetBayes["situacao"])
datasetBayes["qsemana1"]                       = le.fit_transform(datasetBayes["qsemana1"])
datasetBayes["qsemana2"]                       = le.fit_transform(datasetBayes["qsemana2"])
datasetBayes["qsemana3"]                       = le.fit_transform(datasetBayes["qsemana3"])
datasetBayes["qsemana4"]                       = le.fit_transform(datasetBayes["qsemana4"])
datasetBayes["qsemana5"]                       = le.fit_transform(datasetBayes["qsemana5"])
datasetBayes["qsemana6"]                       = le.fit_transform(datasetBayes["qsemana6"])
datasetBayes["qsemana7"]                       = le.fit_transform(datasetBayes["qsemana7"])
datasetBayes["qsemana8"]                       = le.fit_transform(datasetBayes["qsemana8"])
datasetBayes["manha"]                          = le.fit_transform(datasetBayes["manha"])
datasetBayes["tarde"]                          = le.fit_transform(datasetBayes["tarde"])
datasetBayes["noite"]                          = le.fit_transform(datasetBayes["noite"])

datasetBayes.head()

In [0]:
data = datasetBayes
target = datasetBayes["situacao"]
data.drop(columns=["situacao"],inplace=True)
data.head(2)

In [0]:
#import the necessary module
from sklearn.model_selection import train_test_split
#split data set into train and test sets
data_train, data_test, target_train, target_test = train_test_split(data,target, test_size = 0.30, random_state = 10)

In [0]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
#create an object of the type GaussianNB
gnb = GaussianNB()
#train the algorithm on training data and predict using the testing data
pred = gnb.fit(data_train, target_train).predict(data_test)
#print(pred.tolist())
#print the accuracy score of the model
print("Naive-Bayes accuracy : ",accuracy_score(target_test, pred, normalize = True))

In [0]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
#create an object of type LinearSVC
svc_model = LinearSVC(random_state=11)
#train the algorithm on training data and predict using the testing data
pred = svc_model.fit(data_train, target_train).predict(data_test)
#print the accuracy score of the model
print("LinearSVC accuracy : ",accuracy_score(target_test, pred, normalize = True))

In [0]:
#import necessary modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#create object of the lassifier
neigh = KNeighborsClassifier(n_neighbors=3)
#Train the algorithm
neigh.fit(data_train, target_train)
# predict the response
pred = neigh.predict(data_test)
# evaluate accuracy
print ("KNeighbors accuracy score : ",accuracy_score(target_test, pred))

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(gnb, classes=["REPROVADO", "APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(svc_model, classes=["REPROVADO", "APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(neigh, classes=["REPROVADO","APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

# Naive Bayes, Regressão Linear e KNN com dias que teve submissão por semana

In [0]:
datasetBayes=table
datasetBayes=datasetBayes.drop(columns=["tituloProva2","tituloProva1","idAluno","ano","matricula","idTurma","igualACeml45","igualACeml123","qsub4","totalsub","qsubp2","qsubp1","qsub5","qsub3","qsub2","qsub1","mediaProvas","quantidadeProva2","questoesFeitasProva2","notaProva2","quantidadeProva1","questoesFeitasProva1","notaProva1","noite","tarde","manha","madrugada","qsemana1","qsemana2","qsemana3","qsemana4","qsemana5","qsemana6","qsemana7","qsemana8"])
datasetBayes.head(1)

In [0]:
#import the necessary module
from sklearn import preprocessing
# create the Labelencoder object
le = preprocessing.LabelEncoder()
#convert the categorical columns into numeric
datasetBayes["situacao"]                       = le.fit_transform(datasetBayes["situacao"])
datasetBayes["submeteu1"]                      = le.fit_transform(datasetBayes["submeteu1"])
datasetBayes["submeteu2"]                      = le.fit_transform(datasetBayes["submeteu2"])
datasetBayes["submeteu3"]                      = le.fit_transform(datasetBayes["submeteu3"])
datasetBayes["submeteu4"]                      = le.fit_transform(datasetBayes["submeteu4"])
datasetBayes["submeteu5"]                      = le.fit_transform(datasetBayes["submeteu5"])
datasetBayes["submeteu6"]                      = le.fit_transform(datasetBayes["submeteu6"])
datasetBayes["submeteu7"]                      = le.fit_transform(datasetBayes["submeteu7"])
datasetBayes["submeteu8"]                      = le.fit_transform(datasetBayes["submeteu8"])

datasetBayes.head()

In [0]:
data = datasetBayes
target = datasetBayes["situacao"]
data.drop(columns=["situacao"],inplace=True)
data.head(2)

In [0]:
#import the necessary module
from sklearn.model_selection import train_test_split
#split data set into train and test sets
data_train, data_test, target_train, target_test = train_test_split(data,target, test_size = 0.30, random_state = 11)

In [0]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
#create an object of the type GaussianNB
gnb = GaussianNB()
#train the algorithm on training data and predict using the testing data
pred = gnb.fit(data_train, target_train).predict(data_test)
#print(pred.tolist())
#print the accuracy score of the model
print("Naive-Bayes accuracy : ",accuracy_score(target_test, pred, normalize = True))

In [0]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
#create an object of type LinearSVC
svc_model = LinearSVC(random_state=0)
#train the algorithm on training data and predict using the testing data
pred = svc_model.fit(data_train, target_train).predict(data_test)
#print the accuracy score of the model
print("LinearSVC accuracy : ",accuracy_score(target_test, pred, normalize = True))

In [0]:
#import necessary modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#create object of the lassifier
neigh = KNeighborsClassifier(n_neighbors=3)
#Train the algorithm
neigh.fit(data_train, target_train)
# predict the response
pred = neigh.predict(data_test)
# evaluate accuracy
print ("KNeighbors accuracy score : ",accuracy_score(target_test, pred))

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(gnb, classes=["REPROVADO", "APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(gnb, classes=["REPROVADO", "APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(neigh, classes=["REPROVADO","APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

# Naive Bayes, Regressão Linear e KNN com dias que teve submissão por semana e turno em que aluno submeteu

In [0]:
datasetBayes=table
datasetBayes=datasetBayes.drop(columns=["tituloProva2","tituloProva1","idAluno","ano","matricula","idTurma","igualACeml45","igualACeml123","qsub4","totalsub","qsubp2","qsubp1","qsub5","qsub3","qsub2","qsub1","mediaProvas","quantidadeProva2","questoesFeitasProva2","notaProva2","quantidadeProva1","questoesFeitasProva1","notaProva1","qsemana1","qsemana2","qsemana3","qsemana4","qsemana5","qsemana6","qsemana7","qsemana8"])
datasetBayes.head(1)

In [0]:
#import the necessary module
from sklearn import preprocessing
# create the Labelencoder object
le = preprocessing.LabelEncoder()
#convert the categorical columns into numeric
datasetBayes["situacao"]                       = le.fit_transform(datasetBayes["situacao"])
datasetBayes["submeteu1"]                      = le.fit_transform(datasetBayes["submeteu1"])
datasetBayes["submeteu2"]                      = le.fit_transform(datasetBayes["submeteu2"])
datasetBayes["submeteu3"]                      = le.fit_transform(datasetBayes["submeteu3"])
datasetBayes["submeteu4"]                      = le.fit_transform(datasetBayes["submeteu4"])
datasetBayes["submeteu5"]                      = le.fit_transform(datasetBayes["submeteu5"])
datasetBayes["submeteu6"]                      = le.fit_transform(datasetBayes["submeteu6"])
datasetBayes["submeteu7"]                      = le.fit_transform(datasetBayes["submeteu7"])
datasetBayes["submeteu8"]                      = le.fit_transform(datasetBayes["submeteu8"])
datasetBayes["madrugada	"]                     = le.fit_transform(datasetBayes["madrugada"])
datasetBayes["manha"]                          = le.fit_transform(datasetBayes["manha"])
datasetBayes["tarde"]                          = le.fit_transform(datasetBayes["tarde"])
datasetBayes["noite"]                          = le.fit_transform(datasetBayes["noite"])

datasetBayes.head()

In [0]:
data = datasetBayes
target = datasetBayes["situacao"]
data.drop(columns=["situacao"],inplace=True)
data.head(2)

In [0]:
#import the necessary module
from sklearn.model_selection import train_test_split
#split data set into train and test sets
data_train, data_test, target_train, target_test = train_test_split(data,target, test_size = 0.30, random_state =11)

In [0]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
#create an object of the type GaussianNB
gnb = GaussianNB()
#train the algorithm on training data and predict using the testing data
pred = gnb.fit(data_train, target_train).predict(data_test)
#print(pred.tolist())
#print the accuracy score of the model
print("Naive-Bayes accuracy : ",accuracy_score(target_test, pred, normalize = True))

In [0]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
#create an object of type LinearSVC
svc_model = LinearSVC(random_state=0)
#train the algorithm on training data and predict using the testing data
pred = svc_model.fit(data_train, target_train).predict(data_test)
#print the accuracy score of the model
print("LinearSVC accuracy : ",accuracy_score(target_test, pred, normalize = True))

In [0]:
import necessary modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#create object of the lassifier
neigh = KNeighborsClassifier(n_neighbors=3)
#Train the algorithm
neigh.fit(data_train, target_train)
# predict the response
pred = neigh.predict(data_test)
# evaluate accuracy
print ("KNeighbors accuracy score : ",accuracy_score(target_test, pred))

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(gnb, classes=["REPROVADO", "APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(svc_model, classes=["REPROVADO", "APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(neigh, classes=["REPROVADO","APROVADO"])
visualizer.fit(data_train, target_train) # Fit the training data to the visualizer
visualizer.score(data_test, target_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

# Todos os campos pra serem treinados

In [0]:
#import the necessary module
from sklearn import preprocessing
# create the Labelencoder object
le = preprocessing.LabelEncoder()
#convert the categorical columns into numeric
datasetBayes["notaProva1"]                     = le.fit_transform(datasetBayes["notaProva1"])
datasetBayes["questoesFeitasProva1"]           = le.fit_transform(datasetBayes["questoesFeitasProva1"])
datasetBayes["quantidadeProva1"]               = le.fit_transform(datasetBayes["quantidadeProva1"])
datasetBayes["notaProva2"]                     = le.fit_transform(datasetBayes["notaProva2"])
datasetBayes["questoesFeitasProva2"]           = le.fit_transform(datasetBayes["questoesFeitasProva2"])
datasetBayes["quantidadeProva2"]               = le.fit_transform(datasetBayes["quantidadeProva2"])
datasetBayes["mediaProvas"]                    = le.fit_transform(datasetBayes["mediaProvas"])
datasetBayes["situacao"]                       = le.fit_transform(datasetBayes["situacao"])
datasetBayes["qsub1"]                          = le.fit_transform(datasetBayes["qsub1"])
datasetBayes["qsub2"]                          = le.fit_transform(datasetBayes["qsub2"])
datasetBayes["qsub3"]                          = le.fit_transform(datasetBayes["qsub3"])
datasetBayes["qsub4"]                          = le.fit_transform(datasetBayes["qsub4"])
datasetBayes["qsub5"]                          = le.fit_transform(datasetBayes["qsub5"])
datasetBayes["qsubp1"]                         = le.fit_transform(datasetBayes["qsubp1"])
datasetBayes["qsubp2"]                         = le.fit_transform(datasetBayes["qsubp2"])
datasetBayes["totalsub"]                       = le.fit_transform(datasetBayes["totalsub"])
datasetBayes["igualACeml123"]                  = le.fit_transform(datasetBayes["igualACeml123"])
datasetBayes["igualACeml45"]                   = le.fit_transform(datasetBayes["igualACeml45"])
datasetBayes["qsemana1"]                       = le.fit_transform(datasetBayes["qsemana1"])
datasetBayes["qsemana2"]                       = le.fit_transform(datasetBayes["qsemana2"])
datasetBayes["qsemana3"]                       = le.fit_transform(datasetBayes["qsemana3"])
datasetBayes["qsemana4"]                       = le.fit_transform(datasetBayes["qsemana4"])
datasetBayes["qsemana5"]                       = le.fit_transform(datasetBayes["qsemana5"])
datasetBayes["qsemana6"]                       = le.fit_transform(datasetBayes["qsemana6"])
datasetBayes["qsemana7"]                       = le.fit_transform(datasetBayes["qsemana7"])
datasetBayes["qsemana8"]                       = le.fit_transform(datasetBayes["qsemana8"])
datasetBayes["submeteu1"]                      = le.fit_transform(datasetBayes["submeteu1"])
datasetBayes["submeteu2"]                      = le.fit_transform(datasetBayes["submeteu2"])
datasetBayes["submeteu3"]                      = le.fit_transform(datasetBayes["submeteu3"])
datasetBayes["submeteu4"]                      = le.fit_transform(datasetBayes["submeteu4"])
datasetBayes["submeteu5"]                      = le.fit_transform(datasetBayes["submeteu5"])
datasetBayes["submeteu6"]                      = le.fit_transform(datasetBayes["submeteu6"])
datasetBayes["submeteu7"]                      = le.fit_transform(datasetBayes["submeteu7"])
datasetBayes["submeteu8"]                      = le.fit_transform(datasetBayes["submeteu8"])
datasetBayes["madrugada	"]                     = le.fit_transform(datasetBayes["madrugada"])
datasetBayes["manha"]                          = le.fit_transform(datasetBayes["manha"])
datasetBayes["tarde"]                          = le.fit_transform(datasetBayes["tarde"])
datasetBayes["noite"]                          = le.fit_transform(datasetBayes["noite"])

datasetBayes.head()